#Inital Mucking Around with BERT

To add additional vocab, see https://github.com/google-research/bert/issues/9

In [88]:
#place unzipped BERT model files in same level dir
#https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

import os
UPLOAD_TIME = '2018_10_18' #@param {type:"string"}
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_PRETRAINED_DIR = os.path.realpath(BERT_MODEL)
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))

***** BERT pretrained directory: E:\Jupyter\keras-bert\getting_it_running\uncased_L-12_H-768_A-12 *****


In [4]:
import os
import json
from keras_bert.loader import load_trained_model_from_checkpoint
from keras_bert.bert import *
config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True)

model.summary(line_length=120)

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               (None, 512)                0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             (None, 512)                0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 512, 768), (30522, 23440896      Input-Token[0][0]                       
________________________________________________________________________________________________________________________
Embedding-Segment (Embedding)   

In [5]:
model.compile('adam', 'sparse_categorical_crossentropy')

In [7]:
dict_path = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
dict_path

'E:\\Jupyter\\keras-bert\\getting_it_running\\uncased_L-12_H-768_A-12\\vocab.txt'

In [104]:
import codecs
bsz = 8
MASK = ['[MASK]']
CLS = ['[CLS]']
SEP = ['[SEP]']
UNK = ['[UNK]']

tokens = CLS + 'VALVE'.lower().split() + 'GLOBE LESS EQ'.lower().split() + MASK + 'SEAL 0.75 IN 2173# \
PER ANSI B16.34-1981 FORGED SSBODY BOLTED BONNET, SOCKET WELD'.lower().split() + SEP + 'MUST COMPLY WITH ASME B&PV CODE, \
SECT III, CL 2, 1974 ED W75 ADD AND SPEC CMTR FOR BODY (SA182 TP 316L), INDICATOR TUBE (SA479 GR 316), \
BONNET TUBE (SA479 GR XM-19), MAIN DISC (SA564 GR 630), AND FIXED CORE (A276 GR 410 OR A479 GR 410), \
NOREM 02A HARD FACING'.lower().split() + SEP + 'MUST PROVIDE CERTIFICATION TO EQUIP QUALIFICATION AND SEISMIC REQUIREMENTS PER \
SPEC, PT/MT FOR HARD FACING/OVERLAY'.lower().split() + SEP

#develop
tokens = CLS + 'Attempting to'.lower().split() + MASK + 'inventions he could patent and market, \
Tesla conducted a range of experiments with mechanical oscillators generators \
electrical discharge tubes and early X-ray imaging'.lower().split() + SEP + \
'He also built a wireless controlled boat , one of the first ever exhibited'.lower().split() + SEP

#cleared
tokens = CLS + 'ON 01/04/2019 at 01:29 The Control Room received alarm window 5B02D , PPS TROUBLE / GND ,\
and RJ point SBYS36, Plant Protection System Channel D Ground. The alarm locked in for 45 minutes \
then'.lower().split() + MASK +\
'. The alarm response was followed and no abnormalities were identified'.lower().split() + SEP + 'The ground alarm light on the\
front of PPS Cabinet D is ON which is normal , meaning there is no ground indicated'.lower().split() + SEP + 'Wrote CR to\
request support from I&C'.lower().split() + SEP

#cleared, None:There was no word there
tokens = CLS + 'ON 01/04/2019 at 01:29 The Control Room received alarm window 5B02D , PPS TROUBLE / GND ,\
and RJ point SBYS36, Plant Protection System Channel D Ground. The alarm locked in for 45 minutes \
then'.lower().split() + MASK +\
'. The alarm response was followed and no'.lower().split() + MASK +  'abnormalities were identified'.lower().split() + SEP + 'The ground alarm light on the\
front of PPS Cabinet D is ON which is normal , meaning there is no ground indicated'.lower().split() + SEP + 'Wrote CR to\
request support from I&C'.lower().split() + SEP

#cleared, abnormalities
tokens = CLS + 'ON 01/04/2019 at 01:29 The Control Room received alarm window 5B02D , PPS TROUBLE / GND ,\
and RJ point SBYS36, Plant Protection System Channel D Ground. The alarm locked in for 45 minutes \
then'.lower().split() + MASK +\
'. The alarm response was followed and no'.lower().split() + MASK +  'were identified'.lower().split() + SEP + 'The ground alarm light on the\
front of PPS Cabinet D is ON which is normal , meaning there is no ground indicated'.lower().split() + SEP + 'Wrote CR to\
request support from I&C'.lower().split() + SEP

print(tokens)

token_dict = {}
with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
token_dict_rev = {v: k for k, v in token_dict.items()}

def token_lookup(token):
    try:
        x = token_dict[token]
        return x
    except KeyError:
        return token_dict['[UNK]']
    
token_input = np.asarray([[token_lookup(token) for token in tokens] + [0] * (512 - len(tokens)) for i in range(bsz)])
seg_input = np.asarray([[0] * len(tokens) + [0] * (512 - len(tokens)) for i in range(bsz)])
mask_input = np.asarray([[0, 1, 1] + [0] * (512 - 3) for i in range(bsz)])

['[CLS]', 'on', '01/04/2019', 'at', '01:29', 'the', 'control', 'room', 'received', 'alarm', 'window', '5b02d', ',', 'pps', 'trouble', '/', 'gnd', ',and', 'rj', 'point', 'sbys36,', 'plant', 'protection', 'system', 'channel', 'd', 'ground.', 'the', 'alarm', 'locked', 'in', 'for', '45', 'minutes', 'then', '[MASK]', '.', 'the', 'alarm', 'response', 'was', 'followed', 'and', 'no', '[MASK]', 'were', 'identified', '[SEP]', 'the', 'ground', 'alarm', 'light', 'on', 'thefront', 'of', 'pps', 'cabinet', 'd', 'is', 'on', 'which', 'is', 'normal', ',', 'meaning', 'there', 'is', 'no', 'ground', 'indicated', '[SEP]', 'wrote', 'cr', 'torequest', 'support', 'from', 'i&c', '[SEP]']


In [105]:
print(token_input[0][:len(tokens)])
predicts = model.predict([token_input, seg_input, mask_input])[0]
predicts = np.argmax(predicts, axis=-1)
print(predicts[0][:len(tokens)])
print(list(map(lambda x: token_dict_rev[x], predicts[0][1:len(tokens)])))

[  101  2006   100  2012   100  1996  2491  2282  2363  8598  3332   100
  1010   100  4390  1013   100   100   100  2391   100  3269  3860  2291
  3149  1040   100  1996  8598  5299  1999  2005  3429  2781  2059   103
  1012  1996  8598  3433  2001  2628  1998  2053   103  2020  4453   102
  1996  2598  8598  2422  2006   100  1997   100  5239  1040  2003  2006
  2029  2003  3671  1010  3574  2045  2003  2053  2598  5393   102  2626
 13675   100  2490  2013   100   102]
[1012 2006  100 2012  100 1996 2491 2282 2363 8598 3332  100 1010  100
  100 1013  100  100  100 2391  100 3269 3860 2291 3149 1040  100 1996
 8598 5299 1999 2005 3429 2781 2059 3030 1012 1996 8598 3433 2001 2628
 1998 2053 3471 2020 4453 1012 1996 2598 8598 2422 2006  100 1997  100
 5239 1040 2003 2006 2029 2003 3671 1010 3574 2045 2003 2053 2598 1012
 1012  100 1024  100 2490 2013 1012 1012]
['on', '[UNK]', 'at', '[UNK]', 'the', 'control', 'room', 'received', 'alarm', 'window', '[UNK]', ',', '[UNK]', '[UNK]', '/', '[